In [ ]:

from google.colab import drive
drive.mount('/content/drive')


# Standard Libraries
import os
import string
import re
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.auto import tqdm


# Importing NLTK for natural language processing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')



# Installing langid library for language identification in text
!pip install -q update langid
import langid



Mounted at /content/drive


______________________________________________________________________________________________________
## **Data loading**
______________________________________________________________________________________________________

In [ ]:
# loading the seattle dataset files for preparing it for futher pipeline
review_datapath = "/content/drive/MyDrive/Colab Notebooks/research/airbnb_review_data.csv"
property_datapath = "/content/drive/MyDrive/Colab Notebooks/research/main_data/description_property.csv"

review_data = pd.read_csv(review_datapath)
property_data = pd.read_csv(property_datapath)


In [ ]:

review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84849 entries, 0 to 84848
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  84849 non-null  int64 
 1   user_id      84849 non-null  int64 
 2   review       84831 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [ ]:
# Droping rows with NaN values in the 'review' column
review_data.dropna(subset=['review'], inplace=True)

review_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 84831 entries, 0 to 84848
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  84831 non-null  int64 
 1   user_id      84831 non-null  int64 
 2   review       84831 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.6+ MB


In [ ]:
property_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3818 entries, 0 to 3817
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   property_id  3818 non-null   int64  
 1   description  3818 non-null   object 
 2   ratings      3171 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 89.6+ KB


In [ ]:
# Calculating the number of missing values for confirmation

# For review_data
missing_values_review = review_data.isnull().sum()
print(f"Missing values in review_data:\n{'='*40}\n", missing_values_review)
print(f"\n{'|'*100}\n")

# For property_data
missing_values_property = property_data.isnull().sum()
print(f"Missing values in property_data:\n{'='*40}\n", missing_values_property)


Missing values in review_data:
 property_id    0
user_id        0
review         0
dtype: int64

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

Missing values in property_data:
 property_id      0
description      0
ratings        647
dtype: int64


In [ ]:

review_data.head()


,property_id,user_id,review
0,7202016,28943674,Cute and cozy place. Perfect location to every...
1,7202016,32440555,Kelly has a great room in a very central locat...
2,7202016,37722850,"Very spacious apartment, and in a great neighb..."
3,7202016,33671805,Close to Seattle Center and all it has to offe...
4,7202016,34959538,Kelly was a great host and very accommodating ...


In [ ]:

property_data.head()


,property_id,description,ratings
0,241032,Make your self at home in this charming one-be...,95.0
1,953595,Chemically sensitive? We've removed the irrita...,96.0
2,3308979,New modern house built in 2013. Spectacular s...,97.0
3,7421966,A charming apartment that sits atop Queen Anne...,NaN
4,278830,Cozy family craftman house in beautiful neighb...,92.0


In [ ]:

def get_review_stats(review_text):
    """
      Calculate statistics for a single review.

      Input:
          review_text (str): review text to analyze.

      Output:
          dict:
              - "Sentences": number of sentences in the review.
              - "Max Sentence Length": length of the longest sentence.
              - "Min Sentence Length": length of the shortest sentence.
    """
    # Tokenizing the review text into sentences
    sentences = nltk.sent_tokenize(review_text)

    # Counting the number of sentences
    num_sentences = len(sentences)

    # Calculating sentence lengths
    sentence_lengths = [len(nltk.word_tokenize(sent)) for sent in sentences]

    # Calculating the maximum and minimum sentence lengths
    max_sentence_length = max(sentence_lengths)
    min_sentence_length = min(sentence_lengths)

    return {
        "Sentences": num_sentences,
        "Max Sentence Length": max_sentence_length,
        "Min Sentence Length": min_sentence_length,
    }

def calculate_aggregate_stats(reviews):
    """
      Calculate aggregate statistics for a list of reviews.

      Input:
          reviews (list): list of reviews to analyze.

      Output:
          None
    """
    num_reviews = len(reviews)
    total_sentences = 0
    total_words = 0
    min_sentences = float('inf')
    max_sentences = 0
    min_sentence_length = float('inf')
    max_sentence_length = 0
    min_sentence_review = None
    max_sentence_review = None

    for review in reviews:
        if not isinstance(review, str):
            continue  # Skip non-string elements
        stats = get_review_stats(review)

        # Extracting individual stats
        num_sentences = stats["Sentences"]
        max_sentence_length = stats["Max Sentence Length"]
        min_sentence_length = stats["Min Sentence Length"]

        # Update total counts
        total_sentences += num_sentences

        # Update min and max sentence counts
        if num_sentences < min_sentences:
            min_sentences = num_sentences
            min_sentence_review = review
        if num_sentences > max_sentences:
            max_sentences = num_sentences
            max_sentence_review = review

        # Update min and max sentence lengths
        if min_sentence_length < min_sentence_length:
            min_sentence_length = min_sentence_length
        if max_sentence_length > max_sentence_length:
            max_sentence_length = max_sentence_length

    # Calculating average sentence length
    average_sentence_length = total_words / total_sentences if total_sentences > 0 else 0

    print("Aggregate Statistics:")
    print(f"Minimum Sentences: {min_sentences} (in review: {min_sentence_review})")
    print(f"Maximum Sentences: {max_sentences} (in review: {max_sentence_review})")
    print(f"Average Sentence Length: {average_sentence_length}")
    print(f"Minimum Sentence Length: {min_sentence_length}")
    print(f"Maximum Sentence Length: {max_sentence_length}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

reviews_review_data = review_data['review']

# obtaining  the stats of 'review' using calculate_aggregate_stats()
calculate_aggregate_stats(reviews_review_data)

Aggregate Statistics:
Minimum Sentences: 1 (in review: Very welcoming and a nicer place to live in the Seattle area)
Maximum Sentences: 55 (in review: Pros: Residence is close to the University District; a quiet neighborhood.   Both bedrooms have ample closet space.  We slept in the master bedroom. The extra blankets and pillows made the bed very comfortable.  The nightstand's lamps gave a bright light for reading.  The bathroom is large and towels are provided.  The apartment is heated with space heaters.  They were all set when we arrived and really kept the area warm.   The kitchen has some very nice cookware and cutlery.   Many eateries are within walking distance.  A no frills place - good value for the space.  Laura promptly responded to emails and phone calls. 


Cons:   Low ceilings in some areas of the apartment.  The floor, except in the bathroom, is covered with black foam matting used underneath exercise equipment.   We kept tripping over the "bubble” where it does not lay 

In [ ]:

description_property_data = property_data['description']

# obtaining  the stats of 'description' using calculate_aggregate_stats()
calculate_aggregate_stats(description_property_data)


Aggregate Statistics:
Minimum Sentences: 1 (in review: By Special Request Only)
Maximum Sentences: 36 (in review: Welcome Bienvenue Willkommen Bienvenida ようこそ  欢迎  בברכה  ترحيب Secure building in quiet neighborhood 2.5 miles to UW. Off street parking. 5 min walk to public transit. Best for short term singles, grad students, young professionals, mature vacationers. LGBT welcome. Verified off line ID required for pre-approval. Guest Room: .  Pretty space filled with light,  art, and books.  .  Leather love seat with twin hide-a-bed and new memory foam mattress.   .  Best suited for a single person, but two people can manage if they don't mind tight quarters.  Couples may choose from one full size or pair twin size AERO beds on floor. .  Two large drawers store personal items. .  Hall closet for hats and coats. .  Wall hooks and hangers. .  Under bed basket storage. .  Cable TV .  Air Conditioning .  WIFI Full Private Guest Bath: .  Basic toiletries  .  Hair dryer .  Bathrobe for women.  

______________________________________________________________________________________________________
## **Detecting & Filtering Non_English Textual data:**
______________________________________________________________________________________________________

**_It was found there were some non-english reviews. This cause disruption in topic modelling. Since they are really less in the number, we will be removing Non-English reviews_**

In [ ]:
pd.set_option('display.max_colwidth', None)

# Sample of some non-english reviews
reviews_for_desired_user_id = review_data.loc[review_data['user_id'] == 14362449, 'review']

print(reviews_for_desired_user_id)


386    Sean und seine Frau Clara sind sehr freundliche, aufmerksame und hilfsbereite Gastgeber. Sie bringen ihren Gästen großes Vertrauen entgegen. Schlafzimmer und Badezimmer waren gepflegt und großzügig. Als Mitbenutzer der Küche und des Wohnzimmers fühlten wir uns willkommen. Sean ist so freundlich, mein Fahrrad in seiner Garage aufzubewahren, solange ich durch Kanada reise. Ich kann diese Gastgeber nur weiter empfehlen. Uwe Gerischer
Name: review, dtype: object


In [ ]:


def preprocess_text(text):
    """
    Preprocesses text by converting it to lowercase, removing punctuation, stopwords, and detecting language.

    Input:
                     text (type:str):     The input text to be preprocessed.

    Output(type:str):
                     If the detected language is English, the preprocessed text is returned.
                     If the detected language is non-English, None is returned.
    """
    CONFIDENCE_THRESHOLD = 0.99
    SHORT_TEXT_THRESHOLD = 7

    try:
        # Converting to lowercase
        text = text.lower()
        # Removing punctuation
        text = ''.join([char for char in text if char not in string.punctuation])
        # Removing stopwords
        stop_words = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word not in stop_words])
        # Removing numbers
        text = ''.join([char for char in text if not char.isdigit()])

    except Exception as e:

        print("Error occurred for text:", text)
        print("Error message:", str(e))
        return None

    #  Language identification
    lang, confidence = langid.classify(text)

    # Checking if the language is English or if the confidence is below the threshold
    if lang == 'en' :#or confidence < CONFIDENCE_THRESHOLD:
        return text

    else:
        return None



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84831 entries, 0 to 84848
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  84831 non-null  int64 
 1   user_id      84831 non-null  int64 
 2   review       84831 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.6+ MB


In [ ]:
text_column = 'review'

# Applying language identification and preprocessing
review_data['processed_review'] = review_data[text_column].apply(preprocess_text)

# Filtering rows where 'processed_review' is None
Extracted_non_english_review = review_data[review_data['processed_review'].isna()]

In [ ]:
Extracted_non_english_review

,property_id,user_id,review,processed_review
0,7202016,38110731,"The place was really nice, clean, and the most important aspect; it was close to everything so we moved across the city easily.\nMuy limpio y espacioso, Kelly fue muy agradable y lo mas importante es que esta cerca de prácticamente todo lo que nos permitió movernos fácilmente por la cuidad.",None
1,1205666,11033019,ohh～Sean's home is amazing！！！i never seen that beautiful dream home before！！！and that are so friendly and cleaning～～ if i come to Seattle again～～i will rent it again～～,None
2,1205666,14362449,"Sean und seine Frau Clara sind sehr freundliche, aufmerksame und hilfsbereite Gastgeber. Sie bringen ihren Gästen großes Vertrauen entgegen. Schlafzimmer und Badezimmer waren gepflegt und großzügig. Als Mitbenutzer der Küche und des Wohnzimmers fühlten wir uns willkommen. Sean ist so freundlich, mein Fahrrad in seiner Garage aufzubewahren, solange ich durch Kanada reise. Ich kann diese Gastgeber nur weiter empfehlen. Uwe Gerischer",None
3,1205666,18033144,"Es war wundervoll bei Sean und seiner Familie. Das Zimmer ist perfekt von der Größe und wir haben uns sehr willkommen gefühlt. Es waren zwei sehr schöne Tage und wir hätten gern noch mehr Zeit gehabt. Wir konnten die Küche und das Wohnzimmer mitnutzen, alles ist sehr geräumig, hell und freundlich. Mit dem Bus kommt man gut in die Stadt, das Haus ist ruhig gelegen, so dass man gut schlafen kann.\r\nEs wird sogar ein kleines Frühstück mit Kaffee oder Tee geboten, was man sich selbst zurecht machen kann. Sean und seine Frau sind sehr freundlich und offen, Gespräche kommen leicht zustande. Ich kann also diese Übernachtung nur empfehlen.\r\nFür Auto-Reisende ist ein Parkplatz direkt vor der Tür.",None
4,1205666,28611673,"Sean, Carla & Sohn waren wirklich ganz tolle Gastgeber. Es waren unsere ersten zwei Nächte mit einer Buchung via AIRBNB und wir sind so zufrieden gewesen, dass wir gerne wiederkommen möchten oder auch noch anderer Gastgeber und Städte dadurch kennenlernen möchten. Wir haben vor unserer Anreise mehrmals per eMail miteinander geschrieben und nach unserer Landung in Seattle hatten wir direkt den Kontakt um zu erfahren, dass wir früher als geplant anreisen konnten. Uns wurde dann der Schlüssel übergeben, das Zimmer und Badzimmer gezeigt sowie alle Bereiche im Hause gezeigt, die wir mitbenutzen durften.",None
...,...,...,...,...
770,6079131,37336305,숙소가 깔끔하고 아늑했다. 주변이 조용해 휴식을 취하기에 가장 적절한 장소였다. 집 앞에 버스 정류장이 바로 있어서 교통에도 불편함이 없었다. 또 오고싶은 그런 숙소다.,None
771,6079131,28649034,"Fue agradable, los anfitriones preparan tu estancia con detalles para el propósito de tu viaje. Tenian guias de la ciudad, lo esencial en la cocina que no venden en pequeñas cantidades: aceite, sal, jabones, cafe.... En fin están pendientes de sus huéspedes pero son muy discretos .",None
772,264829,2283990,"Great location (close to light rail), quiet neighborhood.",None
773,4577542,25805389,"Die Unterkunft war sehr schön und genau so, wie auf den Fotos und wie beschrieben. Die Wohnung war geschmackvoll und eingerichtet und es war alles vorhanden, was für einen schönen Aufentahlt nötig ist. Sie lag ruhig im hinteren Bereich des Hauses und der private Garten war wie eine Oase in der Großstadt.\r\nDie zentrale Lage war sehr gut und man war schnell zu Fuß bei vielen schönen Restaurants und Shops. Capitol Hill ist ein wirklich toller Stadtteil um Seattle und seine Bewohner zu erleben und um seinen Aufenthalt zu geniesen. Die Erreichbarkeit mit dem Auto war auch ohne Probleme und sehr schnell möglich. DownTown war auch schnell und einfach zu erreichen.\r\nWährend unserem Aufenthalt war Jessica sehr bemüht. Nachdem die Wettervorhersage ""heiße Tage"" mitteilte lies Jessica uns sogar eine Klimaanlage bringen. \r\nWir würden diese Unterkunft jederzeit unseren Freunden und Bekannten empfehlen und sofort wieder buchen.\r\nEin Großen Dank an Jessica",None


In [ ]:
Extracted_non_english_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 775 entries, 0 to 774
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   property_id       775 non-null    int64 
 1   user_id           775 non-null    int64 
 2   review            775 non-null    object
 3   processed_review  9 non-null      object
dtypes: int64(2), object(2)
memory usage: 24.3+ KB


In [ ]:
# Saving the dataframe and going through it manually to remove english comment from it
Extracted_non_english_review.to_csv('/content//Extracted_non_english_review.csv', index=False)



In [ ]:
# loading the dataframe after manually removing
inspected_filtered_non_english_review = pd.read_csv('/content/non_english_df_airbnb_review.csv')
inspected_filtered_non_english_review

,Unnamed: 0,property_id,user_id,review,processed_review
0,9,7202016,38110731,NaN,NaN
1,369,1205666,11033019,NaN,NaN
2,386,1205666,14362449,"Sean und seine Frau Clara sind sehr freundliche, aufmerksame und hilfsbereite Gastgeber. Sie bringen ihren Gästen großes Vertrauen entgegen. Schlafzimmer und Badezimmer waren gepflegt und großzügig. Als Mitbenutzer der Küche und des Wohnzimmers fühlten wir uns willkommen. Sean ist so freundlich, mein Fahrrad in seiner Garage aufzubewahren, solange ich durch Kanada reise. Ich kann diese Gastgeber nur weiter empfehlen. Uwe Gerischer",NaN
3,404,1205666,18033144,"Es war wundervoll bei Sean und seiner Familie. Das Zimmer ist perfekt von der Größe und wir haben uns sehr willkommen gefühlt. Es waren zwei sehr schöne Tage und wir hätten gern noch mehr Zeit gehabt. Wir konnten die Küche und das Wohnzimmer mitnutzen, alles ist sehr geräumig, hell und freundlich. Mit dem Bus kommt man gut in die Stadt, das Haus ist ruhig gelegen, so dass man gut schlafen kann.\r\nEs wird sogar ein kleines Frühstück mit Kaffee oder Tee geboten, was man sich selbst zurecht machen kann. Sean und seine Frau sind sehr freundlich und offen, Gespräche kommen leicht zustande. Ich kann also diese Übernachtung nur empfehlen.\r\nFür Auto-Reisende ist ein Parkplatz direkt vor der Tür.",NaN
4,458,1205666,28611673,"Sean, Carla & Sohn waren wirklich ganz tolle Gastgeber. Es waren unsere ersten zwei Nächte mit einer Buchung via AIRBNB und wir sind so zufrieden gewesen, dass wir gerne wiederkommen möchten oder auch noch anderer Gastgeber und Städte dadurch kennenlernen möchten. Wir haben vor unserer Anreise mehrmals per eMail miteinander geschrieben und nach unserer Landung in Seattle hatten wir direkt den Kontakt um zu erfahren, dass wir früher als geplant anreisen konnten. Uns wurde dann der Schlüssel übergeben, das Zimmer und Badzimmer gezeigt sowie alle Bereiche im Hause gezeigt, die wir mitbenutzen durften.",NaN
...,...,...,...,...,...
752,84117,6079131,37336305,숙소가 깔끔하고 아늑했다. 주변이 조용해 휴식을 취하기에 가장 적절한 장소였다. 집 앞에 버스 정류장이 바로 있어서 교통에도 불편함이 없었다. 또 오고싶은 그런 숙소다.,NaN
753,84121,6079131,28649034,"Fue agradable, los anfitriones preparan tu estancia con detalles para el propósito de tu viaje. Tenian guias de la ciudad, lo esencial en la cocina que no venden en pequeñas cantidades: aceite, sal, jabones, cafe.... En fin están pendientes de sus huéspedes pero son muy discretos .",NaN
754,84197,264829,2283990,NaN,NaN
755,84378,4577542,25805389,"Die Unterkunft war sehr schön und genau so, wie auf den Fotos und wie beschrieben. Die Wohnung war geschmackvoll und eingerichtet und es war alles vorhanden, was für einen schönen Aufentahlt nötig ist. Sie lag ruhig im hinteren Bereich des Hauses und der private Garten war wie eine Oase in der Großstadt.\r\nDie zentrale Lage war sehr gut und man war schnell zu Fuß bei vielen schönen Restaurants und Shops. Capitol Hill ist ein wirklich toller Stadtteil um Seattle und seine Bewohner zu erleben und um seinen Aufenthalt zu geniesen. Die Erreichbarkeit mit dem Auto war auch ohne Probleme und sehr schnell möglich. DownTown war auch schnell und einfach zu erreichen.\r\nWährend unserem Aufenthalt war Jessica sehr bemüht. Nachdem die Wettervorhersage ""heiße Tage"" mitteilte lies Jessica uns sogar eine Klimaanlage bringen. \r\nWir würden diese Unterkunft jederzeit unseren Freunden und Bekannten empfehlen und sofort wieder buchen.\r\nEin Großen Dank an Jessica",NaN


In [ ]:
# Computing accuracy
num_of_detected_non_english_text = len(inspected_filtered_non_english_review)
num_true_non_english_reviews = inspected_filtered_non_english_review['review'].count()

accuracy = num_true_non_english_reviews / num_of_detected_non_english_text
print("Accuracy of Langid:", accuracy)

Accuracy of Langid: 0.8943196829590488


In [ ]:
# Dropping the 'processed_text' column
inspected_filtered_non_english_review.drop(columns=['processed_text'], inplace=True)

# Remove rows with NaN values in the 'review' column to only have the non_english reviews
inspected_filtered_non_english_review.dropna(subset=['review'], inplace=True)

inspected_filtered_non_english_review

,Unnamed: 0,property_id,user_id,review
2,386,1205666,14362449,"Sean und seine Frau Clara sind sehr freundliche, aufmerksame und hilfsbereite Gastgeber. Sie bringen ihren Gästen großes Vertrauen entgegen. Schlafzimmer und Badezimmer waren gepflegt und großzügig. Als Mitbenutzer der Küche und des Wohnzimmers fühlten wir uns willkommen. Sean ist so freundlich, mein Fahrrad in seiner Garage aufzubewahren, solange ich durch Kanada reise. Ich kann diese Gastgeber nur weiter empfehlen. Uwe Gerischer"
3,404,1205666,18033144,"Es war wundervoll bei Sean und seiner Familie. Das Zimmer ist perfekt von der Größe und wir haben uns sehr willkommen gefühlt. Es waren zwei sehr schöne Tage und wir hätten gern noch mehr Zeit gehabt. Wir konnten die Küche und das Wohnzimmer mitnutzen, alles ist sehr geräumig, hell und freundlich. Mit dem Bus kommt man gut in die Stadt, das Haus ist ruhig gelegen, so dass man gut schlafen kann.\r\nEs wird sogar ein kleines Frühstück mit Kaffee oder Tee geboten, was man sich selbst zurecht machen kann. Sean und seine Frau sind sehr freundlich und offen, Gespräche kommen leicht zustande. Ich kann also diese Übernachtung nur empfehlen.\r\nFür Auto-Reisende ist ein Parkplatz direkt vor der Tür."
4,458,1205666,28611673,"Sean, Carla & Sohn waren wirklich ganz tolle Gastgeber. Es waren unsere ersten zwei Nächte mit einer Buchung via AIRBNB und wir sind so zufrieden gewesen, dass wir gerne wiederkommen möchten oder auch noch anderer Gastgeber und Städte dadurch kennenlernen möchten. Wir haben vor unserer Anreise mehrmals per eMail miteinander geschrieben und nach unserer Landung in Seattle hatten wir direkt den Kontakt um zu erfahren, dass wir früher als geplant anreisen konnten. Uns wurde dann der Schlüssel übergeben, das Zimmer und Badzimmer gezeigt sowie alle Bereiche im Hause gezeigt, die wir mitbenutzen durften."
6,537,4681885,39867100,"Notre hôte a été très amicale. Comme nous avons loué pour la fin de semaine et elle était absente pendant la fin de semaine, nous l'avons rencontré un peu avant et un peu après. En fin de compte, nous avons eu sa maison au complet pour nous. Son chien, un Labrador, a été adorable.\r\nLe logement a été tel que indiqué. Nous nous sommes servis des appareils dans la cuisine et de la laveuse et la secheuse.\r\nUn quartier tranquile, typiquement bourgeoisie moyenne."
7,604,2420536,34506602,"这是我们一家三口第一次来美国，第一次在Airbnb定房间，对于什么都是第一次的我们，Tom给了我们非常热情的接待，房间比照片更好，为我们准备了很多零食，超大浴室为我们的旅途解除了疲劳，为我们的旅程提供了很多建设性的意见，知道我们爱吃米饭，特意为我们准备了电饭锅。房子的地理位置也很好，去哪里都很方便，还可以去华盛顿大学和湖边散步，住了五天，每天行程很满，离开时很舍不得，临走前Tom知道我儿子喜欢玩扑克，把珍藏多年的一大箱扑克拿出来给我们欣赏，并且让孩子自己选一副扑克送给他，儿子高兴得不得了！总之，这次美国之行非常成功，感谢Tom，期待下次再来西雅图!\nAt first,my English is not well,please don't laugh at me.\nThis is the first time come to the United States, all for the first time in an room, all of thing is the first time to us, Tom gave us a very warm reception, room is better than photos,he have prepared a lot of snacks, very large bathroom for our journey to remove the fatigue, for our journey provides many constructive opinions,he know that we love to eat rice, specially prepared rice cooker for us.\nLocation of the house is very good, very convenient to go anywhere, you can also go to the university of Washington and the lake for a walk, stayed there for five days, daily schedule is very full, very loathe to give up when we leave.\nTom knew my son likes to play poker, he bring out a trunk of poker that he have collect for years to our appreciation, let my son choose a pack of poker !My son was so happy.\nIn short, the trip to the United States was very successful, thank you, Tom, look forward to Seattle next time."
...,...,...,...,...
751,84071,442487,28347118,"Margot è sempre stata gentile e disponibile. La casa è confortevole, ampia, silenziosa, in un quartiere molto gradevole ben servito dai mezzi pubblici"
752,84117,6079131,37336305,숙소가 깔끔하고 아늑했다. 주변이 조용해 휴식을 취하기에 가장 적절한 장소였다. 집 앞에 버스 정류장이 바로 있어서 교통에도 불편함이 없었다. 또 오고싶은 그런 숙소다.
753,84121,6079131,28649034,"Fue agradable

**After identifying multiple occurrences of the same user commenting in a non-English language, it has been decided to remove rows with their user_id.**

In [ ]:
# Get unique user_ids of non-english reviewers
unique_non_english_reviewers_user_id = inspected_filtered_non_english_review['user_id'].unique()

len(unique_non_english_reviewers_user_id)

731

In [ ]:
# Removing rows with 'user_id' in unique_non_english_reviewers_user_id and dropping the 'processed_text' column
review_data = review_data[~review_data['user_id'].isin(unique_non_english_reviewers_user_id)].drop(columns=['processed_review'])

# Displaying modified DataFrame
review_data


,property_id,user_id,review
0,7202016,28943674,Cute and cozy place. Perfect location to everything!
1,7202016,32440555,"Kelly has a great room in a very central location. \r\nBeautiful building , architecture and a style that we really like. \r\nWe felt guite at home here and wish we had spent more time.\r\nWent for a walk and found Seattle Center with a major food festival in progress. What a treat.\r\nVisited the Space Needle and the Chihuly Glass exhibit. Then Pikes Place Market. WOW. Thanks for a great stay."
2,7202016,37722850,"Very spacious apartment, and in a great neighborhood. This is the kind of apartment I wish I had!\r\n\r\nDidn't really get to meet Kelly until I was on my out, but she was always readily available by phone. \r\n\r\nI believe the only ""issue"" (if you want to call it that) was finding a place to park, but I sincerely doubt its easy to park anywhere in a residential area after 5 pm on a Friday"
3,7202016,33671805,"Close to Seattle Center and all it has to offer - ballet, theater, museum, Space Needle, restaurants of all ilk just blocks away, and the Metropolitan (probably the coolest grocer you'll ever find). Easy to find and Kelly was warm, welcoming, and really interesting to talk to."
4,7202016,34959538,Kelly was a great host and very accommodating in a great neighborhood. She has some great coffee and while I wasn't around much during my stay the time I spent interacting with her was very pleasant. \r\n\r\nThe apartment is in a great location and very close to the Seattle Center. The neighborhood itself has a lot of good food as well!
...,...,...,...
84843,3624990,37578535,"Comfortability upon arrival and everything within a walking distance! The three of us settled in quickly and had plenty of options of where to sleep. All of the furniture was more than comfortable and it all seemed quite clean. No complaints on our end, we truly were spoiled here. We even got to see the nice veiw from the roof! All in all a great trip of seeing Seattle, we would definitely book a trip again!"
84844,3624990,37419458,"The description and pictures of the apartment were exactly what we received. Moreover, the place was very nice and we really enjoyed our stay! The location was perfect for being near to the conference center. In addition, we stayed during a weird stretch of weather (sometimes hot, sometimes cold), and they were very accommodating by providing both extra heating and cooling units. Check-in was smooth and thorough. We appreciated receiving 6 sets of keys, since we had 4 adults staying in the same place. Also, we forgot to return one of the parking fobs when we checked out, and they let us return it late without an issue. All together, a very enjoyable experience and we would stay here again!"
84845,3624990,6933252,"We had an excellent stay. It was clean and comfortable, and very convenient to the convention center and downtown. The beds were comfy, the apartment was quiet, I would stay there again any time."
84847,3624990,24445024,"Very good apartement, clean and well sized. Situated next to the convension center, take the back entrance and you will be there in no time. In walking distance to most everything downtown, and close to good places like the six arms just 100yrds up the street. on the negative side can two of the rooms be some what noisy due laundry in the building."


## **Iteration 1**
______________________________________________________________________________________________________
**_Repeating the process to check if there are more complete non-english reviews_**

In [ ]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84037 entries, 0 to 84848
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  84037 non-null  int64 
 1   user_id      84037 non-null  int64 
 2   review       84037 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.6+ MB


In [ ]:
text_column = 'review'

# Applying language identification and preprocessing
review_data['processed_review'] = review_data[text_column].apply(preprocess_text)

# Filtering rows where 'processed_review' is None
Extracted_non_english_review_1 = review_data[review_data['processed_review'].isna()]

In [ ]:
Extracted_non_english_review_1

,property_id,user_id,review,processed_review
0,7735100,40603512,我们是一家三口，可爱的女儿，夫妻二人都是中国来的访问学者，来到美丽的西雅图，住在了Roger的房子里，房子位于美丽的艺术小镇fremont，有各种各样的雕塑，还有宇宙中心的路标，各种小店的橱窗也是充满特色，富有艺术气息。安静美丽的小镇，充满了秋天的味道。离西雅图市区比较近。房子位置好，能从窗户里看到Rainer山。很不幸我们没有看到。,None
1,7550234,40619236,Sehr gut,None
2,7550234,47521443,房间的描述与实际相符，离华盛顿西雅图分校很近，非常完美，干净且安静，wifi很好，早餐也比较丰盛，房间温馨舒适。户主为客人保持了良好的私人空间，下次会再次光临！,None
3,136480,21869632,房子很整洁，位置如果没有车可能比较不方便。房东人很好，家里有只性格温顺也不认生的小猫咪。不过，床真的有点小，对两个人来说。建议房东给房门安装一个可以锁的门锁。,None
4,2420536,34373179,房东Tom热情友善，服务周到；房间整洁卫生，设施齐全；house位置挺好，公交方便，便于停车，是个不错的住处，值得选择。下次如还有机会，还会再次入住的。,None
...,...,...,...,...
368,3552523,29119658,色々とご迷惑おかけしてしまいました。部屋はとても良く快適でした。街までも近くて、良かったです。ありがとうございました。,None
369,4030600,30620728,整體環境讓人覺得舒服，乾凈。房東是個很體貼的人，會站在房客的角度思考。唯一可惜的是這次旅行沒遇到房東，但那並不影響整個旅行的氛圍，希望下次有機會可以見面。,None
370,4030600,27952981,房东人不错。准备了咖啡和点心等。房间一般，有股味道。,None
371,3394837,35271435,wonderful！ perfect！\n,None


In [ ]:
Extracted_non_english_review_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   property_id       373 non-null    int64 
 1   user_id           373 non-null    int64 
 2   review            373 non-null    object
 3   processed_review  19 non-null     object
dtypes: int64(2), object(2)
memory usage: 11.8+ KB


In [ ]:
# Saving the dataframe and going through it manually to remove english comment from it
Extracted_non_english_review.to_csv('/content//Extracted_non_english_review_1.csv', index=False)



In [ ]:
# Loading the inspected_filtered_non_english_review DataFrame
inspected_filtered_non_english_review_1 = pd.read_csv('/content/non_english_df_new.csv')
inspected_filtered_non_english_review_1

,Unnamed: 0,property_id,user_id,review,processed_review
0,9,7202016,38110731,NaN,NaN
1,80,7735100,40603512,我们是一家三口，可爱的女儿，夫妻二人都是中国来的访问学者，来到美丽的西雅图，住在了Roger的房子里，房子位于美丽的艺术小镇fremont，有各种各样的雕塑，还有宇宙中心的路标，各种小店的橱窗也是充满特色，富有艺术气息。安静美丽的小镇，充满了秋天的味道。离西雅图市区比较近。房子位置好，能从窗户里看到Rainer山。很不幸我们没有看到。,NaN
2,288,7550234,40619236,Sehr gut,NaN
3,301,7550234,47521443,房间的描述与实际相符，离华盛顿西雅图分校很近，非常完美，干净且安静，wifi很好，早餐也比较丰盛，房间温馨舒适。户主为客人保持了良好的私人空间，下次会再次光临！,NaN
4,369,1205666,11033019,ohh～Sean's home is amazing！！！i never seen that beautiful dream home before！！！and that are so friendly and cleaning～～ if i come to Seattle again～～i will rent it again～～,NaN
...,...,...,...,...,...
426,83163,3394837,35271435,NaN,NaN
427,83165,2075094,10090016,NaN,NaN
428,83371,442487,24465870,NaN,NaN
429,83499,264829,2283990,NaN,NaN


In [ ]:
# Computing accuracy
num_of_detected_non_english_text_1 = len(inspected_filtered_non_english_review_1)
num_true_non_english_reviews_1 = inspected_filtered_non_english_review_1['review'].count()

accuracy_1 = num_true_non_english_reviews / num_of_detected_non_english_text
print("Accuracy of Langid:", accuracy_1)

Accuracy of Langid: 0.8943196829590488


In [ ]:
# Dropping the 'processed_text' column
inspected_filtered_non_english_review_1.drop(columns=['processed_review'], inplace=True)

# Removing rows with NaN values in the 'review' column to only have the non_english reviews
inspected_filtered_non_english_review_1.dropna(subset=['review'], inplace=True)

inspected_filtered_non_english_review_1

,Unnamed: 0,property_id,user_id,review
1,80,7735100,40603512,我们是一家三口，可爱的女儿，夫妻二人都是中国来的访问学者，来到美丽的西雅图，住在了Roger的房子里，房子位于美丽的艺术小镇fremont，有各种各样的雕塑，还有宇宙中心的路标，各种小店的橱窗也是充满特色，富有艺术气息。安静美丽的小镇，充满了秋天的味道。离西雅图市区比较近。房子位置好，能从窗户里看到Rainer山。很不幸我们没有看到。
2,288,7550234,40619236,Sehr gut
3,301,7550234,47521443,房间的描述与实际相符，离华盛顿西雅图分校很近，非常完美，干净且安静，wifi很好，早餐也比较丰盛，房间温馨舒适。户主为客人保持了良好的私人空间，下次会再次光临！
4,369,1205666,11033019,ohh～Sean's home is amazing！！！i never seen that beautiful dream home before！！！and that are so friendly and cleaning～～ if i come to Seattle again～～i will rent it again～～
5,491,136480,21869632,房子很整洁，位置如果没有车可能比较不方便。房东人很好，家里有只性格温顺也不认生的小猫咪。不过，床真的有点小，对两个人来说。建议房东给房门安装一个可以锁的门锁。
...,...,...,...,...
419,81704,11012,28034432,地理位置优越，离市中心15分钟路程；床铺很舒服，屋内设施很齐全，看的出主人很有心思；6+1儿童虽然有点紧凑但也过得去，适合对狗不过敏的旅客。
420,81983,3552523,29119658,色々とご迷惑おかけしてしまいました。部屋はとても良く快適でした。街までも近くて、良かったです。ありがとうございました。
423,82890,4030600,30620728,整體環境讓人覺得舒服，乾凈。房東是個很體貼的人，會站在房客的角度思考。唯一可惜的是這次旅行沒遇到房東，但那並不影響整個旅行的氛圍，希望下次有機會可以見面。
424,82899,4030600,27952981,房东人不错。准备了咖啡和点心等。房间一般，有股味道。


In [ ]:
# Converting unique_non_english_reviewers_user_id to a set to store all user_ids that have commented in other language
# and Updated unique_non_english_reviewers_set with new_user_ids
unique_non_english_reviewers_user_id_set = set(unique_non_english_reviewers_user_id)
unique_non_english_reviewers_user_id_set.update(inspected_filtered_non_english_review_1['user_id'].unique())



In [ ]:
# Filtered DataFrame based on the unique non-english reviewer user_ids
filtered_non_english_review = review_data[review_data['user_id'].isin(unique_non_english_reviewers_user_id_set)]
filtered_non_english_review

,property_id,user_id,review
80,7735100,40603512,我们是一家三口，可爱的女儿，夫妻二人都是中国来的访问学者，来到美丽的西雅图，住在了Roger...
288,7550234,40619236,Sehr gut
301,7550234,47521443,房间的描述与实际相符，离华盛顿西雅图分校很近，非常完美，干净且安静，wifi很好，早餐也比较...
369,1205666,11033019,ohh～Sean's home is amazing！！！i never seen that...
386,1205666,14362449,Sean und seine Frau Clara sind sehr freundlich...
...,...,...,...
84117,6079131,37336305,숙소가 깔끔하고 아늑했다. 주변이 조용해 휴식을 취하기에 가장 적절한 장소였다. 집...
84121,6079131,28649034,"Fue agradable, los anfitriones preparan tu est..."
84338,7619060,20806545,非常舒适安静的房子，床边有窗户，晚上睡觉可以看见星星月亮，房东非常热情，退房的时候我去机场，...
84378,4577542,25805389,"Die Unterkunft war sehr schön und genau so, wi..."


In [ ]:

len(unique_non_english_reviewers_user_id_set)

864

In [ ]:
# Removing rows with 'user_id' in unique_non_english_reviewers_user_id and drop the 'processed_text' column
review_data = review_data[~review_data['user_id'].isin(unique_non_english_reviewers_user_id_set)].drop(columns=['processed_review'])



In [ ]:
# Display modified DataFrame
review_data


,property_id,user_id,review
0,7202016,28943674,Cute and cozy place. Perfect location to everything!
1,7202016,32440555,"Kelly has a great room in a very central location. \r\nBeautiful building , architecture and a style that we really like. \r\nWe felt guite at home here and wish we had spent more time.\r\nWent for a walk and found Seattle Center with a major food festival in progress. What a treat.\r\nVisited the Space Needle and the Chihuly Glass exhibit. Then Pikes Place Market. WOW. Thanks for a great stay."
2,7202016,37722850,"Very spacious apartment, and in a great neighborhood. This is the kind of apartment I wish I had!\r\n\r\nDidn't really get to meet Kelly until I was on my out, but she was always readily available by phone. \r\n\r\nI believe the only ""issue"" (if you want to call it that) was finding a place to park, but I sincerely doubt its easy to park anywhere in a residential area after 5 pm on a Friday"
3,7202016,33671805,"Close to Seattle Center and all it has to offer - ballet, theater, museum, Space Needle, restaurants of all ilk just blocks away, and the Metropolitan (probably the coolest grocer you'll ever find). Easy to find and Kelly was warm, welcoming, and really interesting to talk to."
4,7202016,34959538,Kelly was a great host and very accommodating in a great neighborhood. She has some great coffee and while I wasn't around much during my stay the time I spent interacting with her was very pleasant. \r\n\r\nThe apartment is in a great location and very close to the Seattle Center. The neighborhood itself has a lot of good food as well!
...,...,...,...
84843,3624990,37578535,"Comfortability upon arrival and everything within a walking distance! The three of us settled in quickly and had plenty of options of where to sleep. All of the furniture was more than comfortable and it all seemed quite clean. No complaints on our end, we truly were spoiled here. We even got to see the nice veiw from the roof! All in all a great trip of seeing Seattle, we would definitely book a trip again!"
84844,3624990,37419458,"The description and pictures of the apartment were exactly what we received. Moreover, the place was very nice and we really enjoyed our stay! The location was perfect for being near to the conference center. In addition, we stayed during a weird stretch of weather (sometimes hot, sometimes cold), and they were very accommodating by providing both extra heating and cooling units. Check-in was smooth and thorough. We appreciated receiving 6 sets of keys, since we had 4 adults staying in the same place. Also, we forgot to return one of the parking fobs when we checked out, and they let us return it late without an issue. All together, a very enjoyable experience and we would stay here again!"
84845,3624990,6933252,"We had an excellent stay. It was clean and comfortable, and very convenient to the convention center and downtown. The beds were comfy, the apartment was quiet, I would stay there again any time."
84847,3624990,24445024,"Very good apartement, clean and well sized. Situated next to the convension center, take the back entrance and you will be there in no time. In walking distance to most everything downtown, and close to good places like the six arms just 100yrds up the street. on the negative side can two of the rooms be some what noisy due laundry in the building."


In [ ]:

review_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 83893 entries, 0 to 84848
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  83893 non-null  int64 
 1   user_id      83893 non-null  int64 
 2   review       83893 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.6+ MB


## **Iteration 2**
______________________________________________________________________________________________________
**_Repeating the process to check if there are more complete non-english reviews_**

In [ ]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 83893 entries, 0 to 84848
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   property_id  83893 non-null  int64 
 1   user_id      83893 non-null  int64 
 2   review       83893 non-null  object
dtypes: int64(2), object(1)
memory usage: 2.6+ MB


In [ ]:
text_column = 'review'

# Applying language identification and preprocessing
review_data['processed_review'] = review_data[text_column].apply(preprocess_text)

# Filter rows where 'processed_review' is None
Extracted_non_english_review_2 = review_data[review_data['processed_review'].isna()]

In [ ]:
Extracted_non_english_review_2

,property_id,user_id,review,processed_review
0,7202016,38110731,"The place was really nice, clean, and the most important aspect; it was close to everything so we moved across the city easily.\nMuy limpio y espacioso, Kelly fue muy agradable y lo mas importante es que esta cerca de prácticamente todo lo que nos permitió movernos fácilmente por la cuidad.",None
1,4681885,16433164,Staying at Kirsten's house was great. \nIt was well located and we didn't have any problem. She was a good host. \n\nRester chez Kirsten était simple. Elle a été une bonne hôte et nous n'avons rencontré aucun problème.,None
2,72743,1762938,"Nice, quiet place.",None
3,72743,4209391,"So many people left excellent comments, as well will I, but I will do it in Chinese to let more of my country folks know it. 這家由Jennifer打理的民宿，整體感覺非常乾淨整潔，而住宅附近也很寧近，會給人一種脫離塵囂的悠然。然而，Jennifer本身則是很好動外向，還喜歡打電動，尤其家裡的55吋大螢幕，打起PS3比到外面的遊藝場還過癮。Jennifer本人就是一個遊客資訊中心，不但會教你怎麼搭車，甚至可能會有免費車票可以送你，我的車票還用不完呢！整體而言，是我從第一眼就喜歡上，而且下次還想再回來的地方，推薦給大家^^",None
4,5319948,28451157,"I stayed at Devin's house over two week for my vacation in Seattle. It was absolutely lovely to stay in his house. Love interior, kitchen, back yard, location, and even his collection of art book! \nEverything was exactly what I expected from pictures. After couple of days of staying, the house made me feel like I'm in real home. Because this is not a hotel, I worried about bathroom ware and kitchen appliance as I was staying long term, but everything what I need was already there. So convenient place. Moreover, the high speed wifi service is just fantastic! As long as Devin's house is on Airbnb, I definitely want to back at his house to enjoying my vacation in Seattle. \n한국인여행객에게\n2주 좀 넘게 이집에 머물렀는데, 프라이버시도 괜찮고, 교통도 편리하고, 근처 노스게이트라는 쇼핑몰도 걸어서 10분 거리입니다. 무엇보다 집 주인이 먼 거리에 있지만 언제든지 연락이 가능해서 머무는데 불편한 점 도 없었습니다.",None
...,...,...,...,...
216,3535171,7706697,Clean quiet affordable. A bock from my nephew's house ...ultra convenient.,None
217,3394837,35271435,wonderful！ perfect！\n,None
218,2075094,10090016,"Annie’s home is located in a nice neighborhood and close to \r\ndowntown with public transit, 5 mins walk to the bus stop and took 15 mins bus to downtown (Pike market, first Starbucks, Art museum, Nordstrom rack, etc.) We even tried to walk to downtown cost 1 hr and 30mins. The bungalow itself was cozy, clean, comfortable with nice view balcony and I loved cooking at her kitchen with her plenty type of spices! This is the first time I slept in a loft bed, it was fun experience for us. Annie herself is a delightful host, she was very communicative and we had great conversation of sharing our travel stories, her true life stories of adventure was unbelievable! We stayed 7 days and I absolutely have no complaints. Highly recommend her lovely bungalow and we’ll definitely stay with her again next time!!\r\nAnnie 的房子在西雅图山上风景很不错的区域，走5分钟到公车站，坐15分钟公车到downtown，我们有一天尝试着走着去downtown用了一个半小时，路上看了Lake union的整个风景。Annie的房子很舒适，装饰的很艺术，阳台的风景很美，厨房里调料很多，住在这里很惬意。房间里是双人的上铺，就是晚上上洗手间有点麻烦，但是这是我第一次睡这样的床，所以还觉得挺有意思的。Annie是个很随意并且热情的姑娘，跟她相处可以很放松，我们一起分享了我们旅行的故事，她的故事都特别的有意思并且很惊险。总之很推荐来西雅图的朋友住在她家！",None
219,442487,24465870,"A relaxing, nice place. Accurate description. Beautiful neighborhood.",None


In [ ]:
# Save the DataFrame to a CSV file
Extracted_non_english_review_2.to_csv('/content/Extracted_non_english_review_2.csv', index=False)



**_After removing previously identified non-English reviews from the original dataset and reevaluating the language detection process, it was noted that despite detecting additional non-English reviews, the overall accuracy persisted at 89.43%._**


 **_After manually checking the review, they contains some non_english reviews but also have same thing in english. So, I decided to keep thoose reviews. Now, saving the updated dataframe to drive for others operation in another pipeline._**

In [ ]:
# saving the filtered df
review_data.to_csv('/content/drive/MyDrive/Colab Notebooks/research/filtered_english_review_extracted_from_airbnb_review_data.csv',index = True)
